In [ ]:
#!pip install transformers torch torchvision timm sentence-transformers pandas numpy scipy

In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image
from tqdm import tqdm
from transformers import BlipProcessor, BlipForConditionalGeneration
from sentence_transformers import SentenceTransformer
from numpy import cov, trace, iscomplexobj
from scipy.linalg import sqrtm

In [2]:
import os

print(os.listdir("kaggle/working/blip-finetuned"))
print("preprocessor_config.json" in os.listdir("kaggle/working/blip-finetuned"))

['checkpoint-8013', 'config.json', 'generation_config.json', 'model.safetensors', 'preprocessor_config.json', 'special_tokens_map.json', 'tokenizer.json', 'tokenizer_config.json', 'vocab.txt']
True


In [3]:
# 1. Fine-Tunnig yapılan BLIP MODELI ve PROCESSOR'I YÜKLE
# Klasör yolları
processor_path = "kaggle/working/blip-finetuned"
model_path = "kaggle/working/blip-finetuned/checkpoint-8013"

# Yükleme
caption_processor = BlipProcessor.from_pretrained(processor_path, local_files_only=True)
caption_model = BlipForConditionalGeneration.from_pretrained(model_path, local_files_only=True)

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


In [ ]:
# Test dosyası ve klasörü
test_csv = 'test.csv'
test_images = 'test/test/'

test_df = pd.read_csv(test_csv)
captions = []

In [ ]:
# RESİMLERDEN CAPTION ÜRET
print("Caption üretimi başlıyor...")
for idx, row in tqdm(test_df.iterrows(), total=len(test_df)):
    image_path = os.path.join("test/test", str(row["image_id"]) + ".jpg")
    image = Image.open(image_path).convert("RGB")
    
    inputs = caption_processor(image, return_tensors="pt")
    out = caption_model.generate(**inputs)
    caption = caption_processor.decode(out[0], skip_special_tokens=True)
    
    captions.append(caption)

Caption üretimi başlıyor...


100%|██████████| 3771/3771 [3:17:12<00:00,  3.14s/it]  


In [ ]:
# SUBMISSION DOSYASI OLUŞTUR
submission_df = pd.DataFrame({
    "image_id": test_df["image_id"],
    "caption": captions
})
submission_df.to_csv("submission.csv", index=False)
print("Captionlar submission.csv dosyasına kaydedildi.")

Captionlar submission.csv dosyasına kaydedildi.
